作业四 文本纠错  
任务描述：给出三个文件，词典库vocab.txt（任何不在词典中出现的词都认为是拼写错误），spell-errors.txt（给出正确单词与常见的拼写错误）与测试数据testdata.txt（1000个存在拼写错误的句子），利用bigram语言模型与Noisy Channel Model进行文本纠错训练。  
步骤（每一步注释标注）：  
一、读取词典库vocab.txt；  
二、对于词库里没有的词，取编辑距离为1和2，生成候选词集合；  
三、读取spell-errors.txt，计算概率p(错误的单词|正确的单词)；  
四、选用nltk语料库movie_reviews，构建bigram语言模型；  
五、读取testdata.txt，找出拼错词，生成候选词，并计算每个候选词的概率；  
六、选出概率最高的候选词，作为修改词。  
注：为便于计算，可采用似然对数，将概率乘积转化为求和。注意平滑。  
输出：对每个句子，输出句子编号，拼错词，修改词（若没有则输出False），候选词列表及概率。最后输出总拼错词数。  
例如：  
第1句：  
protectionst, protectionist  
{'protectionist': -30.484951869432177}  
第2句：  
Tkyo's, False  
第3句：  
retaliation, retaliation  
{'retaliation': -30.97301670912914}  
Japan's, Japan  
{'Japan': -30.748299246173307, 'Japanese': -30.748299246173307}  
……  
Total mistakes: 1351  


In [12]:
dictionary = set(open("vocab.txt").read().split()) #生成字典,统一小写，以免区分大小写
#print(dictionary)

In [27]:
def getWordsSetWithDistanceOne(word): #生成编辑距离为1的候选词
    alphabet = "abcdefghijklmnopqrstuvwxyz"  #字母表
    splits = [(word[:i],word[i:]) for i in range (len(word)+1)] #生成所有分割组
    insert = [a+x+b for a,b in splits for x in alphabet]
    delete = [a+b[1:] for a,b in splits]
    replace = [a+x+b[1:] for a,b in splits for x in alphabet]
    candidate = set(delete+insert+replace)
    return [word for word in candidate if word in dictionary]

def getWordsSetWithDistanceTwo(word): #生成编辑距离为2的候选词
    tmp = getWordsSetWithDistanceOne(word);  #生成距离为1的候选词
    result = []
    for string in tmp:
        result += getWordsSetWithDistanceOne(string)
    return result

def candidate(word):   #候选词集合
    allWords = getWordsSetWithDistanceOne(word)+getWordsSetWithDistanceTwo(word)
    return list(set(allWords))

In [43]:
allErrors = open("spell-errors.txt").read().split("\n")#读取元素
errorDictionary = {}    #最终的存储结果
for Error in allErrors:
    key,val = Error.split(":")  #获取单词与错误列表
    errorList = val.split(",")  #生成错误单词的列表
    probability = {}            #概率词典
    wordsCount = 0;
    for e in errorList:        #个数计数
        if "*" in e:           #errorx*N的格式
            a,b = e.split("*")
            probability[a.strip()] = int(b)
            wordsCount += int(b)
        else:                 #普通格式
            probability[e.strip()] = 1
            wordsCount += 1
        
        for p in probability:
            probability[p] /= wordsCount #将个数转换成频率 
    errorDictionary[key] = probability
#print(errorDictionary)

In [57]:
from nltk.corpus import movie_reviews
comments =movie_reviews.sents(categories=movie_reviews.categories()) #获取所有评论的关键分词
oneWordDict = {}
twoWordsDict = {}

for comment in comments:
    for i in range(len(comment)-1):
        bigram = comment[i] + " " + comment[i+1] 
        #计数
        if comment[i] in oneWordDict:
            oneWordDict[comment[i]] += 1  
        else:
            oneWordDict[comment[i]] = 1
        #计数
        if bigram in twoWordsDict:
            twoWordsDict[bigram] += 1  
        else:
            twoWordsDict[bigram] = 1
wordsOfComments = len(oneWordDict)

In [69]:
import numpy as np
mistakeCount = 0
for line in open("testdata.txt"):
    line = line.split()
    string = line[2:]
    for i in range (len(string)):
        if string[i][-1] in ".,":
            string[i] = string[i][:len(string[i])-1]
    print("第"+line[0]+"句:")
    for word in string:
        #去除末尾的标点
        if word not in dictionary:
            mistakeCount += 1         #错误词汇总数加1
            candidateList = candidate(word)#生成候选词
            
            if(len(candidateList) == 0):#如果不存在候选词
                print(word+","+"False")  
            else:  
                probabilityList = []
                for c in candidateList:
                    #如果这一组(候选-错误)在错误词典中
                    if c in errorDictionary and word in errorDictionary[c]:
                        p = np.log(errorDictionary[c][word])
                    else:
                        p = np.log(1e-6) #等价于log一个极小值
                    
                    #与前一个单词组合
                    pos = string.index(word)
                    if(pos != 0):
                        bigram = string[pos-1]+" "+word;
                    else:
                        bigram = word;
                    if bigram in twoWordsDict:
                        up = twoWordsDict[bigram]+1
                        if pos == 0:
                            down = wordsOfComments
                        elif string[pos-1] not in oneWordDict:
                            down = wordsOfComments
                        else:
                            down = oneWordDict[string[pos-1]]+wordsOfComments
                        p += np.log(up/down)
                    else:
                        p += np.log(1/wordsOfComments)
                    probabilityList.append(p)
                print(word+","+candidateList[probabilityList.index(max(probabilityList))])#输出
                #输出每个候选词的概率
                print("{",end = "")
                for i in range(len(candidateList)):
                    print(candidateList[i]+":"+str(probabilityList[i]),end = "")
                    if(i == len(candidateList)-1):
                        print("}")
                    else:
                        print(",",end="")

第1句:
protectionst,protectionism
{protectionism:-24.406227817296312,protectionist:-24.406227817296312}
第2句:
Tkyo's,False
第3句:
retaiation,retaliation
{retaliation:-24.406227817296312}
Japan's,False
第4句:
tases,taxis
{taxis:-24.406227817296312,cased:-24.406227817296312,case:-24.406227817296312,tiles:-24.406227817296312,stakes:-24.406227817296312,lakes:-24.406227817296312,biases:-24.406227817296312,tusks:-24.406227817296312,capes:-24.406227817296312,taps:-24.406227817296312,vases:-24.406227817296312,wales:-24.406227817296312,tapis:-24.406227817296312,tanks:-24.406227817296312,dales:-24.406227817296312,tale:-24.406227817296312,bases:-24.406227817296312,cages:-24.406227817296312,cases:-24.406227817296312,axes:-24.406227817296312,caves:-24.406227817296312,types:-24.406227817296312,tapes:-24.406227817296312,taken:-24.406227817296312,bales:-24.406227817296312,gashes:-24.406227817296312,taxes:-24.406227817296312,oases:-24.406227817296312,gases:-24.406227817296312,tables:-24.406227817296312,vase:-

{hound:-24.406227817296312,wounds:-24.406227817296312,pound:-24.406227817296312,round:-24.406227817296312,abound:-24.406227817296312,bounds:-24.406227817296312,mount:-24.406227817296312,found:-24.406227817296312,mound:-24.406227817296312,pounds:-24.406227817296312,sound:-24.406227817296312,abounds:-24.406227817296312,mould:-24.406227817296312,sounds:-24.406227817296312,pond:-24.406227817296312,mounds:-24.406227817296312,rounds:-24.406227817296312,boun:-24.406227817296312,fond:-24.406227817296312,bond:-24.406227817296312,fund:-24.406227817296312,hounds:-24.406227817296312,ground:-24.406227817296312,would:-24.406227817296312,bound:-24.406227817296312,around:-24.406227817296312,wound:-24.406227817296312,rotund:-24.406227817296312}
第113句:
bfore,wore
{wore:-24.406227817296312,forge:-24.406227817296312,force:-24.406227817296312,sore:-24.406227817296312,form:-24.406227817296312,bores:-24.406227817296312,for:-24.406227817296312,bore:-24.406227817296312,core:-24.406227817296312,bare:-24.4062278

{estates:-24.406227817296312,statues:-24.406227817296312,stakes:-24.406227817296312,stares:-24.406227817296312,stated:-24.406227817296312,state:-24.406227817296312,states:-24.406227817296312,stages:-24.406227817296312,skates:-24.406227817296312,status:-24.406227817296312}
yetsterday,yesterday
{yesterday:-24.406227817296312}
第215句:
yecterday,yesterday
{yesterday:-24.406227817296312}
第216句:
extenson,extension
{extension:-24.406227817296312,extensions:-24.406227817296312}
第217句:
mocmon,False
第218句:
Affaigs,Affairs
{Affairs:-24.406227817296312,Affaire:-24.406227817296312,affairs:-24.406227817296312,Affaires:-24.406227817296312}
Japan's,False
第219句:
extorps,False
第220句:
Thede,These
{These:-24.406227817296312,Three:-24.406227817296312,Tee:-24.406227817296312,these:-24.406227817296312,there:-24.406227817296312,here:-24.406227817296312,thee:-24.406227817296312,Thea:-24.406227817296312,Tree:-24.406227817296312,The:-24.406227817296312,where:-24.406227817296312,There:-24.406227817296312,Them:-24.

第298句:
tonnas,tonnes
{tonnes:-24.406227817296312,tones:-24.406227817296312,tonne:-24.406227817296312}
第299句:
additiodn,additions
{additions:-24.406227817296312,addiction:-24.406227817296312,addition:-24.406227817296312,audition:-24.406227817296312}
第300句:
peakc,peaks
{peaks:-24.406227817296312,pears:-24.406227817296312,perk:-24.406227817296312,leak:-24.406227817296312,peak:-24.406227817296312,leaks:-24.406227817296312,speaks:-24.406227817296312,leaky:-24.406227817296312,weak:-24.406227817296312,pear:-24.406227817296312,peaky:-24.406227817296312,peas:-24.406227817296312,peck:-24.406227817296312,speak:-24.406227817296312,peals:-24.406227817296312,perky:-24.406227817296312}
第301句:
yiars,pears
{pears:-24.406227817296312,liars:-24.406227817296312,year:-24.406227817296312,fears:-24.406227817296312,gears:-24.406227817296312,ears:-24.406227817296312,yearn:-24.406227817296312,yeard:-24.406227817296312,liar:-24.406227817296312,wears:-24.406227817296312,bears:-24.406227817296312,hears:-24.4062278

{bich:-24.406227817296312,kick:-24.406227817296312,birch:-24.406227817296312,wash:-24.406227817296312,dish:-24.406227817296312,wise:-24.406227817296312,wisp:-24.406227817296312,wits:-24.406227817296312,lick:-24.406227817296312,rich:-24.406227817296312,with:-24.406227817296312,which:-21.758896351228806,pith:-24.406227817296312,itch:-24.406227817296312,hitch:-24.406227817296312,pick:-24.406227817296312,watch:-24.406227817296312,width:-24.406227817296312,tick:-24.406227817296312,wink:-24.406227817296312,wick:-24.406227817296312,rice:-24.406227817296312,bitch:-24.406227817296312,sick:-24.406227817296312,wit:-24.406227817296312,fish:-24.406227817296312,twitch:-24.406227817296312,hick:-24.406227817296312,pitch:-24.406227817296312,witch:-13.991914640994192,wish:-12.382476728560091,ditch:-24.406227817296312,switch:-24.406227817296312}
第360句:
owted,vowed
{vowed:-24.406227817296312,towed:-24.406227817296312,mowed:-24.406227817296312,wed:-24.406227817296312,opted:-24.406227817296312,awed:-24.4062

...

{Excluding:-24.406227817296312,excluding:-24.406227817296312}
quarter's,quarter
{quarter:-24.406227817296312,quarters:-24.406227817296312}
第692句:
increae,increases
{increases:-24.406227817296312,increased:-24.406227817296312,increase:-24.406227817296312}
第693句:
Leonaerdo,Leonardo
{Leonardo:-24.406227817296312,Leonard:-24.406227817296312}
year's,pears
{pears:-24.406227817296312,fears:-24.406227817296312,year:-24.406227817296312,gears:-24.406227817296312,ears:-24.406227817296312,yearn:-24.406227817296312,yeard:-24.406227817296312,wears:-24.406227817296312,bears:-24.406227817296312,hears:-24.406227817296312,years:-24.406227817296312,tears:-24.406227817296312}
第694句:
harvsting,harvesting
{harvesting:-24.406227817296312}
第695句:
sufficienti,sufficient
{sufficient:-24.406227817296312}
第696句:
Brio,trim
{trim:-24.406227817296312,Briton:-24.406227817296312,triol:-24.406227817296312,trio:-24.406227817296312,trip:-24.406227817296312,Brito:-24.406227817296312}
第697句:
betwein,between
{between:-24.40

{wore:-24.406227817296312,shore:-24.406227817296312,he:-24.406227817296312,surf:-24.406227817296312,sores:-24.406227817296312,sur:-24.406227817296312,sue:-24.406227817296312,she:-24.406227817296312,shame:-24.406227817296312,surge:-24.406227817296312,cure:-24.406227817296312,shape:-24.406227817296312,shade:-24.406227817296312,lure:-24.406227817296312,shy:-24.406227817296312,shea:-24.406227817296312,sort:-24.406227817296312,sore:-24.406227817296312,hare:-24.406227817296312,sole:-24.406227817296312,whoe:-24.406227817296312,shone:-24.406227817296312,chore:-24.406227817296312,shed:-24.406227817296312,sharp:-24.406227817296312,shave:-24.406227817296312,bore:-24.406227817296312,core:-24.406227817296312,scare:-24.406227817296312,ore:-24.406227817296312,sloe:-24.406227817296312,shoes:-24.406227817296312,soe:-24.406227817296312,whore:-24.406227817296312,lore:-24.406227817296312,jure:-24.406227817296312,shared:-24.406227817296312,shred:-24.406227817296312,show:-24.406227817296312,spare:-24.406227

第894句:
Morlin,Martin
{Martin:-24.406227817296312,Marin:-24.406227817296312,Marlin:-24.406227817296312,Marvin:-24.406227817296312}
第895句:
posiiton,False
第896句:
aded,add
{add:-24.406227817296312,and:-24.406227817296312,wed:-24.406227817296312,added:-24.406227817296312,famed:-24.406227817296312,raged:-24.406227817296312,aired:-24.406227817296312,ad:-24.406227817296312,faced:-24.406227817296312,aged:-24.406227817296312,waned:-24.406227817296312,hadd:-24.406227817296312,awed:-24.406227817296312,age:-24.406227817296312,faked:-24.406227817296312,aided:-24.406227817296312,aide:-24.406227817296312,waked:-24.406227817296312,adds:-24.406227817296312,aimed:-24.406227817296312,raided:-24.406227817296312,faded:-24.406227817296312,wade:-24.406227817296312,waved:-24.406227817296312,odd:-24.406227817296312,owed:-24.406227817296312,aid:-24.406227817296312,caged:-24.406227817296312,jaded:-24.406227817296312,waded:-24.406227817296312,abed:-24.406227817296312,aber:-24.406227817296312,wadded:-24.40622781729

shaes,shakes
{shakes:-24.406227817296312,seams:-24.406227817296312,shades:-24.406227817296312,stakes:-24.406227817296312,snakes:-24.406227817296312,stares:-24.406227817296312,ships:-24.406227817296312,shame:-24.406227817296312,shamed:-24.406227817296312,shade:-24.406227817296312,sharks:-24.406227817296312,spades:-24.406227817296312,shakers:-24.406227817296312,shows:-24.406227817296312,shaken:-24.406227817296312,shops:-24.406227817296312,shims:-24.406227817296312,sharers:-24.406227817296312,shoes:-24.406227817296312,shards:-24.406227817296312,shaped:-24.406227817296312,shies:-24.406227817296312,shared:-24.406227817296312,shams:-24.406227817296312,shots:-24.406227817296312,shake:-24.406227817296312,share:-24.406227817296312,shares:-24.406227817296312,shines:-24.406227817296312,skies:-24.406227817296312,shapes:-24.406227817296312,hams:-24.406227817296312,sham:-24.406227817296312,shaker:-24.406227817296312,spares:-24.406227817296312,spies:-24.406227817296312,shaded:-24.406227817296312,shoe

Department's,False
government's,governments
{governments:-24.406227817296312,government:-24.406227817296312}
prograem,programs
{programs:-24.406227817296312,program:-24.406227817296312}
prograems,programs
{programs:-24.406227817296312,program:-24.406227817296312}
第998句:
edtimates,estimate
{estimate:-24.406227817296312,estimates:-24.406227817296312,estimated:-24.406227817296312}
chicnek,False
terkeys,turkey
{turkey:-24.406227817296312,turkeys:-24.406227817296312}
第999句:
inspectieon,inspections
{inspections:-24.406227817296312,inspection:-24.406227817296312}
uopltry,False
第1000句:
eventuall,eventual
{eventual:-24.406227817296312,eventually:-24.406227817296312}
coul,cool
{cool:-24.406227817296312,tool:-24.406227817296312,foal:-24.406227817296312,fowl:-24.406227817296312,coral:-24.406227817296312,four:-24.406227817296312,toil:-24.406227817296312,cook:-24.406227817296312,cools:-24.406227817296312,soul:-24.406227817296312,fool:-24.406227817296312,coax:-24.406227817296312,pool:-24.406227817296

In [70]:
print("Total mistakes: "+str(mistakeCount))

Total mistakes: 1347
